## 导入相关包

In [8]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

## 初始化网络：设计网络结构

In [9]:
class MLP2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP2,self).__init__()

        print(' === Initialize a MLP2 Neural Network === ')    

        self.layer1 = nn.Linear(input_dim, hidden_dim)
        
        self.relu = nn.ReLU(inplace=True)

        self.layer2 = nn.Linear(hidden_dim, output_dim)

        print('self.layer1 -- weight:',self.layer1.weight.shape,' bias:',self.layer1.bias.shape )
        print('self.layer2 -- weight:',self.layer2.weight.shape,' bias:',self.layer2.bias.shape )


    def forward(self, x):
        print(' === MLP2 Neural Network Forward Calculation === ')    
        x = torch.tensor(x,dtype=torch.float32)
        print('origin input -- x:',x.shape)

        x1 = self.layer1(x)
        print('after layer 1 -- x1:',x1.shape)

        x1 = self.relu(x1)
        
        x2 = self.layer2(x1)
        print('after layer 2 -- x2:',x2.shape)

        return x2

# hyper parameters
input_dim = 128 # e.g. data input with 128 features
hidden_dim = 64 # you can define your own net structure
output_dim = 8 # e.g. a classification task of 8 class
class_count = 8

# initialize our net
MLP_Net = MLP2(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)


 === Initialize a MLP2 Neural Network === 
self.layer1 -- weight: torch.Size([64, 128])  bias: torch.Size([64])
self.layer2 -- weight: torch.Size([8, 64])  bias: torch.Size([8])


## 初始化数据, 进行前向计算

In [10]:
# initialize input 
batch_size = 5

print(' === Input Tensor Initialization === ')    
input_tensor = torch.ones([batch_size, input_dim])
print('input_tensor:',input_tensor.shape)


print(' === Real Label Initialization === ')    
real_label = torch.randint(low=0,high=8,size= [batch_size] )#3 # 假设该数据实际分类结果应该为第3类
print('real_label:',real_label)

real_one_hot_label  = torch.eye(class_count)[real_label,:]
print('real_one_hot_label:',real_one_hot_label)



 === Input Tensor Initialization === 
input_tensor: torch.Size([5, 128])
 === Real Label Initialization === 
real_label: tensor([0, 5, 6, 7, 2])
real_one_hot_label: tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0.]])


## 进行前向计算

In [11]:
# get output
print(' === Get Output === ')    
output_tensor = MLP_Net(input_tensor)
print('output_tensor:',output_tensor.shape)

 === Get Output === 
 === MLP2 Neural Network Forward Calculation === 
origin input -- x: torch.Size([5, 128])
after layer 1 -- x1: torch.Size([5, 64])
after layer 2 -- x2: torch.Size([5, 8])
output_tensor: torch.Size([5, 8])


## 计算损失函数（target）

In [12]:
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(output_tensor, real_one_hot_label)
print('loss = ',loss)

loss =  tensor(2.1131, grad_fn=<DivBackward1>)


## 计算梯度并更新 Backward

In [13]:
# 定义optimizer
learning_rate = 0.001 # 可自定义学习率，调参
optimizer = optim.Adam(MLP_Net.parameters(), lr=learning_rate)

# 清空梯度，准备计算新的梯度
optimizer.zero_grad()

# backward 利用pytorch自动求导
print('Before backward Calculation')
for name, parms in MLP_Net.named_parameters():	
    print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
        ' -->grad_value:',parms.grad)

loss.backward()

print('After backward Calculation')
for name, parms in MLP_Net.named_parameters():	
    print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
        ' -->grad_value:',parms.grad)
    
# 进行一步更新
optimizer.step()

Before backward Calculation
-->name: layer1.weight -->grad_requirs: True  -->grad_value: None
-->name: layer1.bias -->grad_requirs: True  -->grad_value: None
-->name: layer2.weight -->grad_requirs: True  -->grad_value: None
-->name: layer2.bias -->grad_requirs: True  -->grad_value: None
After backward Calculation
-->name: layer1.weight -->grad_requirs: True  -->grad_value: tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0348, 0.0348, 0.0348,  ..., 0.0348, 0.0348, 0.0348],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
-->name: layer1.bias -->grad_requirs: True  -->grad_value: tensor([ 0.0000,  0.0348,  0.0000,  0.0153, -0.0435,  0.0058,  0.0000, -0.0079,
         0.0180,  0.0129,  0.0000, -0.0071,  0.0000,  0.0000,  0.0079, -0.0220,
       

In [14]:
# 查看更新后的loss  
new_output_tensor = MLP_Net(input_tensor)
loss = loss_fn(new_output_tensor, real_one_hot_label)
print('new loss = ',loss)

 === MLP2 Neural Network Forward Calculation === 
origin input -- x: torch.Size([5, 128])
after layer 1 -- x1: torch.Size([5, 64])
after layer 2 -- x2: torch.Size([5, 8])
new loss =  tensor(2.0584, grad_fn=<DivBackward1>)
